In [1]:
import os
from sklearn.model_selection import train_test_split
import torch
from insightface.app import FaceAnalysis  # Sử dụng InsightFace để phát hiện khuôn mặt và trích xuất đặc trưng
import numpy as np
from torch.nn.functional import cosine_similarity
import pandas as pd
from PIL import Image

/home/quoc14/Code/.venv/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
data_dir = "/home/quoc14/Code/lfw-deepfunneled"

# Dictionary để lưu ảnh của từng người
people_images = {}

# Duyệt qua các thư mục con để tìm danh sách ảnh của từng người
for person_name in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person_name)
    if os.path.isdir(person_dir):
        images = os.listdir(person_dir)
        if len(images) > 1:  # Chỉ chọn những người có hơn 1 ảnh
            people_images[person_name] = [os.path.join(person_dir, img) for img in images]

# Chia train-test cho mỗi người có nhiều hơn 1 ảnh
train_images = []
train_labels = []
test_images = []
test_labels = []

for person_name, images in people_images.items():
    # Chia 80% ảnh cho train và 20% ảnh cho test
    train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)
    
    # Gán nhãn và thêm vào danh sách train/test
    train_images.extend(train_imgs)
    train_labels.extend([person_name] * len(train_imgs))
    
    test_images.extend(test_imgs)
    test_labels.extend([person_name] * len(test_imgs))

# Kết quả
print(f"Tổng số ảnh train: {len(train_images)}")
print(f"Tổng số ảnh test: {len(test_images)}")

Tổng số ảnh train: 6562
Tổng số ảnh test: 2602


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo mô hình buffalo_l từ InsightFace
app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider' if torch.cuda.is_available() else 'CPUExecutionProvider'])
app.prepare(ctx_id=0 if torch.cuda.is_available() else -1)  # ctx_id=0 để sử dụng GPU, -1 để sử dụng CPU


/home/quoc14/Code/.venv/lib/python3.12/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/quoc14/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/quoc14/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/quoc14/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/quoc14/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/quoc14/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [4]:
def extract_face_features(image_path, app):
    # Đọc ảnh và phát hiện khuôn mặt
    img = Image.open(image_path)
    img = np.array(img)
    
    faces = app.get(img)
    if len(faces) == 0:
        print(f"Face detection failed for image: {image_path}")
        return None  # Không tìm thấy khuôn mặt

    # Trích xuất đặc trưng từ khuôn mặt đầu tiên phát hiện được
    features = faces[0].embedding  # Lấy vector đặc trưng của khuôn mặt đầu tiên
    return features

# Tạo cơ sở dữ liệu từ tập train
database = {}
for img_path, label in zip(train_images, train_labels):
    features = extract_face_features(img_path, app)
    if features is not None:
        if label not in database:
            database[label] = []
        database[label].append(features)


/home/quoc14/Code/.venv/lib/python3.12/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Nabil_Shaath/Nabil_Shaath_0002.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Bill_Clinton/Bill_Clinton_0012.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Chan_Gailey/Chan_Gailey_0001.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Lance_Armstrong/Lance_Armstrong_0015.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Gonzalo_Sanchez_de_Lozada/Gonzalo_Sanchez_de_Lozada_0003.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Arsinee_Khanjian/Arsinee_Khanjian_0001.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Jennifer_Aniston/Jennifer_Aniston_0014.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/Tom_Reilly/Tom_Reilly_0002.jpg
Face detection failed for image: /home/quoc14/Code/lfw-deepfunneled/John_Paul_II/John_Paul_II_0004.jpg
Face detection failed for image

In [5]:
cosine_threshold = 0.4  

# Biến để theo dõi kết quả
correct_predictions = 0
false_positive = 0
false_negative = 0
processed_count = 0  # Biến đếm số lượng ảnh đã xử lý
total_test = len(test_images)

# Hàm tính cosine similarity
def cosine_distance(vec1, vec2):
    vec1_tensor = torch.tensor(vec1)
    vec2_tensor = torch.tensor(vec2)
    return cosine_similarity(vec1_tensor.unsqueeze(0), vec2_tensor.unsqueeze(0)).item()

# Đánh giá trên tập test
for img_path, true_label in zip(test_images, test_labels):
    test_features = extract_face_features(img_path, app)
    if test_features is None:
        continue
    
    best_score = -1
    predicted_label = None
    
    for label, vectors in database.items():
        for vec in vectors:
            score = cosine_distance(test_features, vec)
            if score > best_score:
                best_score = score
                predicted_label = label
    
    # So sánh dự đoán với nhãn thực tế dựa trên ngưỡng cosine
    if best_score >= cosine_threshold:
        if predicted_label == true_label:
            correct_predictions += 1
        else:
            false_positive += 1
    else:
        false_negative += 1
    
    # Cập nhật và in số lượng ảnh đã xử lý
    processed_count += 1
    print(f"Đã xử lý {processed_count}/{total_test} ảnh.")

# Tính toán các chỉ số
accuracy = correct_predictions / total_test
fpir = false_positive / total_test
fnir = false_negative / total_test

# Lưu kết quả vào CSV
results = pd.DataFrame({
    "Model": ["ArcFace"],
    "Accuracy": [accuracy],
    "FPIR": [fpir],
    "FNIR": [fnir]
})
results.to_csv("ArcFace_results.csv", index=False)

# In kết quả
print(f"Accuracy: {accuracy}")
print(f"False Positive Identification Rate (FPIR): {fpir}")
print(f"False Negative Identification Rate (FNIR): {fnir}")

Đã xử lý 1/2602 ảnh.
Đã xử lý 2/2602 ảnh.
Đã xử lý 3/2602 ảnh.
Đã xử lý 4/2602 ảnh.
Đã xử lý 5/2602 ảnh.
Đã xử lý 6/2602 ảnh.
Đã xử lý 7/2602 ảnh.
Đã xử lý 8/2602 ảnh.
Đã xử lý 9/2602 ảnh.
Đã xử lý 10/2602 ảnh.
Đã xử lý 11/2602 ảnh.
Đã xử lý 12/2602 ảnh.
Đã xử lý 13/2602 ảnh.
Đã xử lý 14/2602 ảnh.
Đã xử lý 15/2602 ảnh.
Đã xử lý 16/2602 ảnh.
Đã xử lý 17/2602 ảnh.
Đã xử lý 18/2602 ảnh.
Đã xử lý 19/2602 ảnh.
Đã xử lý 20/2602 ảnh.
Đã xử lý 21/2602 ảnh.
Đã xử lý 22/2602 ảnh.
Đã xử lý 23/2602 ảnh.
Đã xử lý 24/2602 ảnh.
Đã xử lý 25/2602 ảnh.
Đã xử lý 26/2602 ảnh.
Đã xử lý 27/2602 ảnh.
Đã xử lý 28/2602 ảnh.
Đã xử lý 29/2602 ảnh.
Đã xử lý 30/2602 ảnh.
Đã xử lý 31/2602 ảnh.
Đã xử lý 32/2602 ảnh.
Đã xử lý 33/2602 ảnh.
Đã xử lý 34/2602 ảnh.
Đã xử lý 35/2602 ảnh.
Đã xử lý 36/2602 ảnh.
Đã xử lý 37/2602 ảnh.
Đã xử lý 38/2602 ảnh.
Đã xử lý 39/2602 ảnh.
Đã xử lý 40/2602 ảnh.
Đã xử lý 41/2602 ảnh.
Đã xử lý 42/2602 ảnh.
Đã xử lý 43/2602 ảnh.
Đã xử lý 44/2602 ảnh.
Đã xử lý 45/2602 ảnh.
Đã xử lý 46/2602 ản